# Raw Data

- prepare OLink and clinical data
- create views on data
- create targets:

event | next 90 days | next 180 days |
--- | --- | --- |
death | `dead90` | `dead180` |
admission to hospital | `adm90`  | `adm180` |

all cases within 90 days will be included into the 180 days, from `incl`usion and from `infl`ammation sample time.

In [ ]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

import seaborn as sns

from lifelines import KaplanMeierFitter

import njab.plotting
from njab.plotting import savefig
import src

import config

In [ ]:
DATA_FOLDER = Path(config.data)
DATA_PROCESSED = Path(config.data_processed)
DATA_PROCESSED.mkdir(exist_ok=True, parents=True)
FOLDER_REPORTS = Path(config.folder_reports) / 'data_prodoc'
FOLDER_REPORTS.mkdir(exist_ok=True, parents=True)

files_out = dict()

config.STUDY_ENDDATE

In [ ]:
DATA_CLINIC = DATA_FOLDER / 'DataSheet - fewer variables_2022-12-12.xlsx'
DATA_OLINK = DATA_FOLDER / 'QC_OlinkProD_wide.tsv'
DATA_OLINK_VAL = DATA_PROCESSED / 'olink_prodoc_val.xlsx'

Load clinical data

In [ ]:
clinic = pd.read_excel(DATA_CLINIC)
clinic.SampleID = clinic.SampleID.str.replace(' ', '')
cols_clinic = src.pandas.get_colums_accessor(clinic)
clinic = clinic.set_index('SampleID').sort_index()

In [ ]:
# clinic
clinic.describe(datetime_is_numeric=True, include='all')

In [ ]:
olink = pd.read_table(DATA_OLINK)
olink = olink.set_index(olink.SampleID.str[4:]).sort_index()
cols_olink = src.pandas.get_colums_accessor(olink)

In [ ]:
# olink
olink.describe(datetime_is_numeric=True, include='all')

In [ ]:
olink_val = pd.read_excel(DATA_OLINK_VAL, index_col=0)
olink_val.index = olink_val.index.str[4:].str.replace(' ', '')
olink_val

## Deaths over time

- one plot with absolute time axis
- one plot relative to diagnosis date

In [ ]:
clinic['dead'] = (clinic['DateDeath'] - clinic['DateInflSample']).notna()
clinic["DateDeath"] = clinic["DateDeath"].fillna(value=config.STUDY_ENDDATE)

In [ ]:
din_a4 = (8.27 * 2, 11.69 * 2)
njab.plotting.make_large_descriptors(32)
fig, ax = plt.subplots(figsize=din_a4)

src.plotting.plot_lifelines(clinic.sort_values('DateInflSample'),
                            start_col='DateInflSample',
                            ax=ax)
_ = plt.xticks(rotation=45)
ax.invert_yaxis()
njab.plotting.set_font_sizes('x-small')
files_out['lifelines'] = FOLDER_REPORTS / 'lifelines.pdf'
savefig(fig, files_out['lifelines'])

In [ ]:
clinic.dead.value_counts()

In [ ]:
fig, axes = plt.subplots(2, sharex=True)
ax = axes[0]
ax.set_yticks([])

ax = clinic.loc[clinic.dead].astype({
    'dead': 'category'
}).plot.scatter(x="DateInflSample",
                y="dead",
                c='blue',
                rot=45,
                ax=ax,
                ylabel='dead')
ax = axes[1]
# ax.axes.yaxis.set_visible(False)
ax.set_yticks([])
ax = clinic.loc[~clinic.dead].astype({
    'dead': 'category'
}).plot.scatter(x="DateInflSample",
                y="dead",
                c='blue',
                rot=45,
                ax=ax,
                ylabel='alive')
_ = fig.suptitle("Inclusion date by survival status")
files_out[
    'death_vs_alive_diagonose_dates'] = FOLDER_REPORTS / 'death_vs_alive_diagonose_dates'
savefig(fig, files_out['death_vs_alive_diagonose_dates'])

In [ ]:
ax = clinic.astype({
    'dead': 'category'
}).plot.scatter(x="DateInflSample",
                y='DateDeath',
                c="dead",
                cmap='Paired',
                rot=45,
                s=3,
                sharex=False)
# ticks = ax.get_xticks()
# ax.set_xticklabels(ax.get_xticklabels(),  horizontalalignment='right')
# ax.set_xticks(ticks)
fontsize = 'xx-small'
min_date, max_date = clinic["DateInflSample"].min(
), clinic["DateInflSample"].max()
ax.plot([min_date, max_date], [min_date, max_date], 'k-', lw=1)
_ = ax.annotate('date', [min_date, min_date + datetime.timedelta(days=20)],
                fontsize=fontsize,
                rotation=25)
offset, rot = 20, 25
delta = 90
_ = ax.plot([min_date, max_date], [
    min_date + datetime.timedelta(days=delta),
    max_date + datetime.timedelta(days=delta)
],
            'k-',
            lw=1)
_ = ax.annotate(f'+ {delta} days',
                [min_date, min_date + datetime.timedelta(days=delta + 20)],
                fontsize=fontsize,
                rotation=25)
delta = 180
ax.plot([min_date, max_date], [
    min_date + datetime.timedelta(days=delta),
    max_date + datetime.timedelta(days=delta)
],
        'k-',
        lw=1)
_ = ax.annotate(f'+ {delta} days',
                [min_date, min_date + datetime.timedelta(days=delta + 20)],
                fontsize=fontsize,
                rotation=25)
delta = 360
ax.plot([min_date, max_date], [
    min_date + datetime.timedelta(days=delta),
    max_date + datetime.timedelta(days=delta)
],
        'k-',
        lw=1)
_ = ax.annotate(f'+ {delta} days',
                [min_date, min_date + datetime.timedelta(days=delta + 20)],
                fontsize=fontsize,
                rotation=25)
fig = ax.get_figure()
files_out[
    'timing_deaths_over_time'] = FOLDER_REPORTS / 'timing_deaths_over_time.pdf'
fig.savefig(files_out['timing_deaths_over_time'])

## Cleanup steps

### Clinic

- [x] encode binary variables (yes, no) as `category`s
  > Be aware that this might cause unexpected behaviour!

Fill derived variables with missing measurements

In [ ]:
clinic.loc[:, clinic.columns.str.contains("Adm")].describe()

In [ ]:
clinic.loc[:, clinic.columns.str.contains("Adm")].sum()

Encode binary variables

In [ ]:
# binary variables
vars_binary = config.clinic_data.vars_binary
clinic[vars_binary].head()

In [ ]:
clinic[vars_binary] = clinic[vars_binary].astype('category')

remaining non numeric variables

In [ ]:
mask_cols_obj = clinic.dtypes == 'object'
clinic.loc[:, mask_cols_obj].describe()

In [ ]:
clinic["HbA1c"] = clinic["HbA1c"].replace(to_replace="(NA)",
                                          value=np.nan).astype(pd.Int32Dtype())
clinic["MaritalStatus"] = clinic["MaritalStatus"].astype('category')
clinic["HeartDiseaseTotal"] = clinic["HeartDiseaseTotal"].replace(
    0, 'no').astype('category')
clinic.loc[:, mask_cols_obj].describe(include='all')

In [ ]:
def get_dummies_yes_no(s, prefix=None):
    return pd.get_dummies(s, prefix=prefix).replace({
        0: 'No',
        1: 'Yes'
    }).astype('category')


clinic = clinic.join(get_dummies_yes_no(clinic["DiagnosisPlace"]))
clinic = clinic.join(
    get_dummies_yes_no(clinic["MaritalStatus"], prefix='MaritalStatus'))
clinic = clinic.join(get_dummies_yes_no(clinic["CauseOfDeath"], prefix='CoD'))
clinic

- few have more than one etiology

In [ ]:
etiology_mask_yes = clinic.loc[:, clinic.columns.str.contains("Eti")] == 'Yes'
etiology_mask_yes.sum(axis=1).value_counts()

In [ ]:
clinic["EtiNonAlco"] = (clinic["EtiAlco"] == 'No') & (etiology_mask_yes.drop(
    'EtiAlco', axis=1).sum(axis=1).astype(bool))
clinic["EtiNonAlco"] = get_dummies_yes_no(clinic["EtiNonAlco"])[True]
clinic["EtiNonAlco"].value_counts()

### Olink

- [x] remove additional meta data
- [x] highlight missing values


In [ ]:
olink.head()

Remove additional metadata

In [ ]:
olink = olink.loc[:, 'IL8':]

Which measurments have missing values

- [ ] Imputation due to limit of detection (LOD) -> how to best impute

In [ ]:
olink.loc[:, olink.isna().any()].describe()

## Timespans

- death only has right censoring, no drop-out
- admission has right censoring, and a few drop-outs who die before their first admission for the cirrhosis

For these individuals, the admission time is censored as the persons died before.

In [ ]:
clinic["DaysToAdmFromInflSample"] = (
    clinic["DateAdm"].fillna(config.STUDY_ENDDATE) -
    clinic["DateInflSample"]).dt.days
clinic["DaysToDeathFromInfl"] = (
    clinic["DateDeath"].fillna(config.STUDY_ENDDATE) -
    clinic["DateInflSample"]).dt.days

cols_clinic = src.pandas.get_colums_accessor(clinic)

cols_view = [
    cols_clinic.DaysToDeathFromInfl,
    cols_clinic.DateAdm,
    # cols_clinic.DateFirstAdmission,
    cols_clinic.DaysToAdmFromInflSample,
    "dead",
    "Adm90",
    "Adm180",
    "Age"
]

mask = (clinic.dead == True) & (clinic.Adm180.isna())
view = clinic.loc[mask, cols_view].sort_values(cols_view)
files_out['died_before_admission'] = FOLDER_REPORTS / 'died_before_adm.xlsx'
view.to_excel(files_out['died_before_admission'])
view

## Kaplan-Meier survival plot

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(clinic["DaysToDeathFromInfl"], event_observed=clinic["dead"])

X_LIMIT = config.MAX_DAYS_INTERVAL

fig, ax = plt.subplots()
y_lim = (0, 1)
ax = kmf.plot(  #title='Kaplan Meier survival curve since inclusion',
    xlim=(0, X_LIMIT),
    ylim=y_lim,
    xlabel='Days since inflammation sample',
    ylabel='survival rate',
    ax=ax,
    legend=False)
ax.legend([f"KP admission (N={clinic['dead'].notna().sum()})", '95% CI'])
_ = ax.vlines(90, *y_lim)
_ = ax.vlines(180, *y_lim)
files_out['km_plot_death'] = FOLDER_REPORTS / 'km_plot_death.pdf'
savefig(fig, files_out['km_plot_death'])

In [ ]:
_ = sns.catplot(x="DaysToDeathFromInfl",
                y="dead",
                hue="DiagnosisPlace",
                data=clinic.astype({'dead': 'category'}),
                height=4,
                aspect=3)
_.set_xlabels('Days from inflammation sample to death or until study end')
ax = _.fig.get_axes()[0]
ylim = ax.get_ylim()
ax.vlines(90, *ylim)
ax.vlines(180, *ylim)
fig = ax.get_figure()
files_out['deaths_along_time'] = FOLDER_REPORTS / 'deaths_along_time.pdf'
savefig(fig, files_out['deaths_along_time'])

## Kaplan-Meier plot admissions

- information was only collected up to 180 days after inflammation sample

In [ ]:
to_exclude = clinic[
    "ExcludeFromAdm (due to mors/terminal just after inclusion)"].notna()

In [ ]:
clinic["LiverAdm180"].value_counts(dropna=False).sort_index()

In [ ]:
to_exclude = clinic[
    "ExcludeFromAdm (due to mors/terminal just after inclusion)"].notna()

In [ ]:
kmf = KaplanMeierFitter()

mask = ~to_exclude
print(f"Based on {mask.sum()} patients")
kmf.fit(clinic.loc[mask, "DaysToAdmFromInflSample"],
        event_observed=clinic.loc[mask, "LiverAdm180"].fillna(0))

fig, ax = plt.subplots()
y_lim = (0, 1)
ax = kmf.plot(  #title='Kaplan Meier curve for liver related admissions',
    xlim=(0, 180),
    ylim=(0, 1),
    xlabel='Days since inflammation sample',
    ylabel='rate no liver-related admission',
    legend=False)
ax.legend([f"KP admission (N={mask.sum()})", '95% CI'])
# _ = ax.vlines(90, *y_lim)
# _ = ax.vlines(180, *y_lim)
fig = ax.get_figure()

files_out['km_plot_admission'] = FOLDER_REPORTS / 'km_plot_admission.pdf'
savefig(fig, files_out['km_plot_admission'])

## Targets

In [ ]:
mask = clinic.columns.str.contains("Adm(90|180)")
clinic.loc[:, mask].describe()  # four targets for liver related admissions

In [ ]:
target_name = {k: f'has{k}' for k in clinic.columns[mask]}
target_name

In [ ]:
targets = {}

for cutoff in [90, 180]:
    targets[f'dead{cutoff:03}infl'] = (clinic["DaysToDeathFromInfl"] <=
                                       cutoff).astype(int)
    targets[f"liverDead{cutoff:03}infl"] = (
        clinic.loc[clinic["CauseOfDeath"] != 'NonLiver',
                   "DaysToDeathFromInfl"] <= cutoff).astype(int)

targets = pd.DataFrame(targets)
targets.describe()

In [ ]:
# to_exclude = clinic["LiverAdm90"].isna() & targets["dead090infl"] == True
# to_exclude.sum()
to_exclude = clinic[
    "ExcludeFromAdm (due to mors/terminal just after inclusion)"].notna()
to_exclude.sum()

In [ ]:
clinic.loc[to_exclude]

In [ ]:
for col_adm, col_death in zip(
    ['Adm180', 'Adm90', 'LiverAdm90', 'LiverAdm180'],
    ['dead180infl', 'dead090infl', 'dead090infl', 'dead180infl']):
    clinic.loc[~to_exclude, col_adm] = clinic.loc[~to_exclude,
                                                  col_adm].fillna(0)
    clinic.loc[to_exclude, col_adm] = np.nan

clinic.loc[:, mask].describe()

In [ ]:
targets = targets.join((clinic.loc[:, mask]).rename(columns=target_name))
for col in target_name.values():
    not_na = targets[col].notna()
    targets.loc[not_na, col] = (targets.loc[not_na, col] > 0).astype(float)
targets = targets.sort_index(axis=1, ascending=False)
targets.describe()

In [ ]:
from src.pandas import combine_value_counts

combine_value_counts(targets)

In [ ]:
ret = []
for var in targets.columns:
    _ = pd.crosstab(targets[var], clinic.DiagnosisPlace)
    _.index = [f'{var.replace("_", " <= ", 1)} - {i}' for i in _.index]
    ret.append(_)
ret = pd.concat(ret)

tab_targets_by_diagnosisPlace = ret
tab_targets_by_diagnosisPlace

add to clinical targets

In [ ]:
clinic = clinic.join(targets)

## Censoring

Date of first Admission is also right-censored

In [ ]:
time_from_inclusion_to_first_admission = clinic["DateAdm"].fillna(
    config.STUDY_ENDDATE) - clinic["DateInflSample"]
time_from_inclusion_to_first_admission.describe()

Who dies without having a first Admission date?

In [ ]:
dead_wo_adm = clinic["DateAdm"].isna() & clinic['dead']
idx_dead_wo_adm = dead_wo_adm.loc[dead_wo_adm].index
print('Dead without admission to hospital:',
      *dead_wo_adm.loc[dead_wo_adm].index)
clinic.loc[dead_wo_adm, ["DateAdm", "DateInflSample", cols_clinic.LiverAdm180]]

## Different overlaps

- save persons with clinical data as potential validation cohort separately
- done after preprocessing of data


In [ ]:
idx_overlap = olink.index.intersection(clinic.index)
idx_overlap

In [ ]:
# in clinical data, but not in olink data
idx_validation = clinic.index.difference(olink.index)
idx_validation = idx_validation.intersection(olink_val.index)
idx_validation

In [ ]:
# in olink data, but not in clinical data -> excluded samples
olink.index.difference(clinic.index)

## Save training cohort separately

In [ ]:
files_out[config.fname_pkl_clinic.stem] = config.fname_pkl_clinic
files_out[config.fname_pkl_olink.stem] = config.fname_pkl_olink
clinic.loc[idx_overlap].to_pickle(config.fname_pkl_clinic)
olink.loc[idx_overlap].to_pickle(config.fname_pkl_olink)

## Save validation cohort separately

In [ ]:
files_out[config.fname_pkl_val_clinic.stem] = config.fname_pkl_val_clinic
clinic.loc[idx_validation].to_pickle(config.fname_pkl_val_clinic)

In [ ]:
# potentially save to yaml to be used elsewhere (or json)
','.join(idx_validation)

## Dump combined data for comparision

In [ ]:
idx_valid_proDoc = [*idx_overlap, *idx_validation]
clinic = clinic.loc[idx_valid_proDoc]
clinic[config.COMPARE_PRODOC] = clinic.index.isin(idx_validation).astype(
    'float')
files_out[config.fname_pkl_prodoc_clinic.stem] = config.fname_pkl_prodoc_clinic
clinic.to_pickle(config.fname_pkl_prodoc_clinic)
olink = pd.concat([olink.loc[idx_overlap], olink_val.loc[idx_validation]])
files_out[config.fname_pkl_prodoc_olink.stem] = config.fname_pkl_prodoc_olink
olink.to_pickle(config.fname_pkl_prodoc_olink)

In [ ]:
files_out[config.fname_pkl_targets.stem] = config.fname_pkl_targets
targets.to_pickle(config.fname_pkl_targets)
clinic[targets.columns].describe()

## Dumped combined clinical data as numeric data for ML applications

In [ ]:
cols_cat = clinic.dtypes == 'category'
cols_cat = clinic.columns[cols_cat]
clinic[cols_cat]

In [ ]:
encode = {
    **{k: 1 for k in ['Yes', 'yes', 'Male']},
    **{k: 0 for k in ['No', 'no', 'Female']}
}
encode

In [ ]:
clinic[cols_cat] = clinic[cols_cat].astype('object').replace(encode)
clinic[cols_cat]

In [ ]:
clinic[cols_cat].describe()

The martial status was made into three dummy variables before (see above):
`MaritalStatus_Divorced, MaritalStatus_Married, MaritalStatus_Relationship, MaritalStatus_Separated, MaritalStatus_Unmarried, MaritalStatus_Widow/widower`

In [ ]:
mask = clinic.dtypes == 'object'
clinic.loc[:, mask]

In [ ]:
mask = clinic.dtypes == 'datetime64[ns]'
clinic.loc[:, mask]

In [ ]:
numeric_cols = (clinic.apply(pd.api.types.is_numeric_dtype))
numeric_cols = clinic.columns[numeric_cols]
clinic[numeric_cols]

In [ ]:
clinic[numeric_cols].dtypes.value_counts()

In [ ]:
to_drop = ["Study ID"]
clinic[numeric_cols].drop(to_drop, axis=1)

In [ ]:
files_out[config.fname_pkl_prodoc_clinic_num.
          stem] = config.fname_pkl_prodoc_clinic_num
clinic[numeric_cols].drop(to_drop,
                          axis=1).to_pickle(config.fname_pkl_prodoc_clinic_num)

## Files saved by notebook

In [ ]:
src.io.print_files(files_out)